# Challenge Three: Testing and Evaluation
## GenAI Delivery Excellence Skills Validation Workshop

**Goal:** Demonstrate the ability to write tests and evaluate responses from large language models.

### Requirements:
1. Create functions that perform specific tasks utilizing Google Gemini
2. Write unit tests to ensure the functions perform as expected
3. Use the Google Evaluation API to evaluate and compare functions using various different prompts

---
### What We're Building:
- **Function 1:** Classifies citizen questions into categories: `Employment`, `General Information`, `Emergency Services`, or `Tax Related`
- **Function 2:** Generates social media posts for government announcements (weather emergencies, holidays, school closings, etc.)
- **Unit Tests:** pytest-based tests for both functions
- **Evaluation:** Google Vertex AI Evaluation Service API to assess and compare prompt strategies

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q google-cloud-aiplatform google-generativeai vertexai pandas ipytest

## Step 2: Configure Google Cloud Project

In [ ]:
import vertexai
import google.generativeai as genai
import pandas as pd
import datetime
import unittest
import ipytest
import pytest

from vertexai.generative_models import GenerativeModel
from vertexai.evaluation import EvalTask

# -------------------------------------------------------
# CONFIGURE: Replace with your Google Cloud project details
# -------------------------------------------------------
PROJECT_ID = "your-project-id"       # e.g. "my-gcp-project-12345"
LOCATION   = "us-central1"
MODEL_ID   = "gemini-1.5-flash-002"  # Latest stable Gemini model

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize the Gemini model (used throughout the notebook)
model = GenerativeModel(MODEL_ID)

print(f"✅ Vertex AI initialized — Project: {PROJECT_ID} | Location: {LOCATION} | Model: {MODEL_ID}")

---
## Part A: Gemini Functions

### Function 1 — `classify_citizen_question(question)`

This function uses Gemini to classify a citizen's question into one of four government service categories:
- `Employment`
- `General Information`
- `Emergency Services`
- `Tax Related`

Because the output is constrained to a fixed set of labels, this is a **deterministic** (classification) task — we can test it with standard unit tests.

In [ ]:
def classify_citizen_question(question: str) -> str:
    """
    Classifies a citizen's question into one of four government service categories
    using Gemini.

    Args:
        question (str): The citizen's question.

    Returns:
        str: One of — 'Employment', 'General Information',
             'Emergency Services', or 'Tax Related'
    """
    prompt = """
You are a government service routing assistant.
Your ONLY job is to classify citizen questions into one of the following categories:

  - Employment
  - General Information
  - Emergency Services
  - Tax Related

Rules:
  1. Output ONLY the category name — no punctuation, no explanation.
  2. If the question could fit multiple categories, choose the MOST specific one.
  3. 'Emergency Services' is reserved for urgent safety/emergency topics only.

Examples:
  Question: How do I apply for unemployment benefits?
  Category: Employment

  Question: My house is on fire, who do I call?
  Category: Emergency Services

  Question: What are the office hours for City Hall?
  Category: General Information

  Question: When is my property tax due?
  Category: Tax Related

Now classify the following:
  Question: {question}
  Category:""".format(question=question)

    response = model.generate_content(prompt)
    # Strip whitespace and return the clean label
    return response.text.strip()


# Quick smoke test
test_questions = [
    "How do I file for workers' compensation?",
    "There is a gas leak on my street!",
    "What documents do I need for a building permit?",
    "How do I appeal my property tax assessment?"
]

print("=== classify_citizen_question — Manual Smoke Test ===")
for q in test_questions:
    result = classify_citizen_question(q)
    print(f"  Q: {q}")
    print(f"  → {result}\n")

### Function 2 — `generate_government_social_media_post(announcement)`

This function uses Gemini to generate a social media post for a government announcement.  
Because the output is free-form text (no single correct answer), this is an **indeterminate** task.  
We will test it using:
1. A Gemini-based **rules checker** (unit test proxy)
2. The **Google Evaluation API** (formal evaluation)

In [ ]:
# -------------------------------------------------------
# Prompt Version A — used as the baseline prompt
# -------------------------------------------------------
SOCIAL_MEDIA_PROMPT_V1 = """
You are the official social media manager for a US state government department.

Your task is to write a single social media post for the following government announcement.

Rules you MUST follow:
  1. Keep the post under 280 characters (Twitter/X limit).
  2. Use a professional but approachable tone suitable for a government account.
  3. Include one relevant hashtag at the end (e.g., #AKGov, #AlaskaAlerts, #SchoolClosure).
  4. Do NOT use emojis.
  5. If it is an emergency, start with: ALERT:

Announcement: {announcement}

Post:"""


# -------------------------------------------------------
# Prompt Version B — improved prompt with few-shot examples
# -------------------------------------------------------
SOCIAL_MEDIA_PROMPT_V2 = """
You are the official social media manager for a US state government department.

Write a social media post for the government announcement below.

Rules you MUST follow:
  1. Keep the post under 280 characters.
  2. Professional but approachable government tone.
  3. Include one relevant hashtag at the end.
  4. No emojis.
  5. Emergency announcements MUST start with: ALERT:

Examples:
  Announcement: All state offices will be closed on Thanksgiving Day, November 28.
  Post: State offices will be closed Thursday, Nov. 28 in observance of Thanksgiving. Normal operations resume Friday, Nov. 29. #AKGov

  Announcement: A major snowstorm is expected tonight. Roads may be impassable.
  Post: ALERT: Major snowstorm expected tonight. Avoid unnecessary travel. Road conditions may be hazardous. Stay safe. #AlaskaAlerts

  Announcement: All public schools in the Anchorage district will be closed tomorrow due to weather.
  Post: Anchorage School District schools are CLOSED tomorrow due to weather. Stay safe and check back for updates. #SchoolClosure

Now write a post for:
  Announcement: {announcement}
  Post:"""


def generate_government_social_media_post(announcement: str, prompt_version: str = "v1") -> str:
    """
    Generates a social media post for a government announcement using Gemini.

    Args:
        announcement (str): The government announcement text.
        prompt_version (str): 'v1' (basic) or 'v2' (few-shot examples).

    Returns:
        str: A social media post string.
    """
    if prompt_version == "v2":
        prompt_template = SOCIAL_MEDIA_PROMPT_V2
    else:
        prompt_template = SOCIAL_MEDIA_PROMPT_V1

    prompt = prompt_template.format(announcement=announcement)
    response = model.generate_content(prompt)
    return response.text.strip()


# Quick smoke test
announcements = [
    "All state offices will be closed on Christmas Day, December 25.",
    "A severe blizzard is forecast for tomorrow. Residents should stay home.",
    "All Fairbanks school district schools will be closed on Monday due to extreme cold."
]

print("=== generate_government_social_media_post — Prompt V1 ===")
for a in announcements:
    post = generate_government_social_media_post(a, prompt_version="v1")
    print(f"  Announcement: {a}")
    print(f"  Post ({len(post)} chars): {post}\n")

---
## Part B: Unit Tests

### Testing Strategy

| Function | Output Type | Testing Method |
|---|---|---|
| `classify_citizen_question` | Deterministic (fixed labels) | Direct `assertEqual` assertions |
| `generate_government_social_media_post` | Indeterminate (free text) | Rules-based LLM checker + Evaluation API |

We use `ipytest` to run `pytest`-style tests directly in the notebook.

In [ ]:
# Configure ipytest to run pytest inside the notebook
ipytest.autoconfig()

### Test Suite 1: `classify_citizen_question` — Deterministic Tests

The valid categories are fixed, so we can assert exact equality.

In [ ]:
%%ipytest -v

# -------------------------------------------------------
# Test Suite 1: Question Classification
# Tests that classify_citizen_question returns the correct
# category for a variety of citizen questions.
# -------------------------------------------------------

VALID_CATEGORIES = {"Employment", "General Information", "Emergency Services", "Tax Related"}

class TestClassifyCitizenQuestion:

    # ---- Employment Questions ----

    def test_unemployment_benefits_is_employment(self):
        """Questions about unemployment should be classified as Employment."""
        result = classify_citizen_question("How do I file for unemployment benefits?")
        assert result == "Employment", f"Expected 'Employment', got '{result}'"

    def test_job_application_is_employment(self):
        """Questions about government job applications should be Employment."""
        result = classify_citizen_question("How do I apply for a state government job?")
        assert result == "Employment", f"Expected 'Employment', got '{result}'"

    def test_workers_comp_is_employment(self):
        """Workers compensation queries should be Employment."""
        result = classify_citizen_question("How do I file a workers' compensation claim?")
        assert result == "Employment", f"Expected 'Employment', got '{result}'"

    # ---- Emergency Services Questions ----

    def test_gas_leak_is_emergency(self):
        """Gas leak reports should be Emergency Services."""
        result = classify_citizen_question("There is a gas leak on my street, who do I call?")
        assert result == "Emergency Services", f"Expected 'Emergency Services', got '{result}'"

    def test_house_fire_is_emergency(self):
        """Fire emergencies should be Emergency Services."""
        result = classify_citizen_question("My neighbor's house is on fire!")
        assert result == "Emergency Services", f"Expected 'Emergency Services', got '{result}'"

    # ---- General Information Questions ----

    def test_office_hours_is_general_info(self):
        """Questions about office hours are General Information."""
        result = classify_citizen_question("What are the hours of operation for the DMV?")
        assert result == "General Information", f"Expected 'General Information', got '{result}'"

    def test_building_permit_is_general_info(self):
        """Permit document questions are General Information."""
        result = classify_citizen_question("What documents do I need to get a building permit?")
        assert result == "General Information", f"Expected 'General Information', got '{result}'"

    # ---- Tax Related Questions ----

    def test_property_tax_is_tax_related(self):
        """Property tax questions should be Tax Related."""
        result = classify_citizen_question("When is my property tax payment due?")
        assert result == "Tax Related", f"Expected 'Tax Related', got '{result}'"

    def test_tax_appeal_is_tax_related(self):
        """Tax assessment appeal questions should be Tax Related."""
        result = classify_citizen_question("How do I appeal my property tax assessment?")
        assert result == "Tax Related", f"Expected 'Tax Related', got '{result}'"

    # ---- Output Validity Check ----

    def test_output_is_always_valid_category(self):
        """Function must ALWAYS return one of the four valid categories."""
        test_inputs = [
            "Where do I pay my water bill?",
            "Is the park open on weekends?",
            "Can I get a job at the city?",
            "I smell gas outside my window."
        ]
        for q in test_inputs:
            result = classify_citizen_question(q)
            assert result in VALID_CATEGORIES, (
                f"Invalid category '{result}' returned for question: '{q}'"
            )

### LLM Rules Checker Helper

Since `generate_government_social_media_post` produces free-form text, we use a second Gemini call to evaluate whether the generated post follows the rules. This is a standard pattern for testing indeterminate LLM outputs.

In [ ]:
def does_post_follow_rules(post: str) -> str:
    """
    Uses Gemini as a judge to check whether a social media post follows
    the government posting rules.

    Rules checked:
      1. Under 280 characters
      2. Professional tone
      3. Contains exactly one hashtag
      4. No emojis

    Returns:
        'Yes' if the post follows all rules, 'No' otherwise.
    """
    prompt = """
Does the following social media post follow ALL of these rules?
  1. It is under 280 characters.
  2. It uses a professional, government-appropriate tone (no slang).
  3. It contains at least one hashtag.
  4. It contains NO emojis.

Answer ONLY 'Yes' or 'No'. Do not explain.

Post: {post}
Answer:""".format(post=post)

    response = model.generate_content(prompt)
    return response.text.strip()


def does_emergency_post_start_with_alert(post: str) -> str:
    """
    Checks specifically whether an emergency post starts with 'ALERT:'.

    Returns:
        'Yes' if the post starts with 'ALERT:', 'No' otherwise.
    """
    # This is deterministic — we can check it directly without an LLM call
    return "Yes" if post.upper().startswith("ALERT:") else "No"


# Quick validation of the rules checker itself
good_post = "State offices closed Dec. 25 for Christmas. Happy holidays from all of us. #AKGov"
bad_post  = "Hey everyone!! 🎄 Offices r closed lol no hashtag here"

print(f"Good post follows rules: {does_post_follow_rules(good_post)}")   # Expected: Yes
print(f"Bad post follows rules:  {does_post_follow_rules(bad_post)}")    # Expected: No

### Test Suite 2: `generate_government_social_media_post` — LLM-Judged Tests

In [ ]:
%%ipytest -v

# -------------------------------------------------------
# Test Suite 2: Social Media Post Generation
# Uses an LLM judge to verify rule compliance, since output
# is indeterminate (no single correct answer).
# -------------------------------------------------------

class TestGenerateGovernmentSocialMediaPost:

    # ---- Rule Compliance Tests ----

    def test_holiday_post_v1_follows_rules(self):
        """Holiday posts (v1 prompt) should follow all government posting rules."""
        post = generate_government_social_media_post(
            "All state offices will be closed on Christmas Day, December 25.",
            prompt_version="v1"
        )
        result = does_post_follow_rules(post)
        assert result == "Yes", f"Post violated rules.\nPost: {post}\nJudge: {result}"

    def test_school_closure_post_v1_follows_rules(self):
        """School closure posts (v1 prompt) should follow all rules."""
        post = generate_government_social_media_post(
            "All Anchorage school district schools will be closed Monday due to extreme cold.",
            prompt_version="v1"
        )
        result = does_post_follow_rules(post)
        assert result == "Yes", f"Post violated rules.\nPost: {post}\nJudge: {result}"

    def test_holiday_post_v2_follows_rules(self):
        """Holiday posts (v2 prompt) should follow all government posting rules."""
        post = generate_government_social_media_post(
            "All state offices will be closed on New Year's Day, January 1.",
            prompt_version="v2"
        )
        result = does_post_follow_rules(post)
        assert result == "Yes", f"Post violated rules.\nPost: {post}\nJudge: {result}"

    # ---- Emergency Post Test ----

    def test_emergency_post_starts_with_alert(self):
        """Emergency posts MUST start with 'ALERT:' per the posting rules."""
        post = generate_government_social_media_post(
            "A severe blizzard warning has been issued. Roads will be impassable tonight.",
            prompt_version="v2"
        )
        result = does_emergency_post_start_with_alert(post)
        assert result == "Yes", f"Emergency post did not start with ALERT:\nPost: {post}"

    # ---- Character Limit Test ----

    def test_post_under_280_characters(self):
        """All generated posts must be under 280 characters."""
        post = generate_government_social_media_post(
            "The annual state fair will be held August 21-31 at the state fairgrounds. Admission is $12.",
            prompt_version="v1"
        )
        assert len(post) <= 280, f"Post exceeds 280 characters ({len(post)} chars):\n{post}"

    # ---- Negative Test: Deliberately bad post ----

    def test_hardcoded_bad_post_fails_rules_check(self):
        """
        Validates that the rules checker correctly identifies
        a manually crafted bad post as rule-violating.
        This is a meta-test to ensure our judge is working.
        """
        bad_post = (
            "🎉🎄 Hey guys!! Offices r closed tomorrow lol enjoy ur day off!! "
            "This message is definitely way too long and informal for a government account and has NO hashtag at all "
            "and also contains multiple emojis which is against the rules we set."
        )
        result = does_post_follow_rules(bad_post)
        assert result == "No", f"Expected rule checker to return 'No' for bad post, got '{result}'"

---
## Part C: Google Evaluation API

### Why Use the Evaluation API?

Unit tests verify rule compliance (pass/fail), but they don't measure **quality**.  
The Vertex AI Evaluation Service measures quality using model-based metrics:

| Metric | What It Measures |
|---|---|
| `fluency` | Natural, grammatically correct language (1–5) |
| `coherence` | Logical, easy-to-follow structure (1–5) |
| `fulfillment` | Does the response completely fulfill the instructions? (1–5) |
| `groundedness` | Does the response stick to the provided context? (0/1) |

We use the Evaluation API to:
1. Evaluate the **social media post generator** using both prompt versions (V1 vs V2)
2. Compare results to determine which prompt produces better quality output

### Step C1: Create the Evaluation Dataset

We create a dataset of government announcements and generate responses from both prompt versions.

In [ ]:
# -------------------------------------------------------
# Evaluation Dataset: Government Announcements
# -------------------------------------------------------

announcements_dataset = [
    "All state government offices will be closed on Thanksgiving Day, November 28th.",
    "A severe winter storm warning is in effect. Residents should avoid travel tonight and tomorrow.",
    "All public schools in the Fairbanks North Star Borough will be closed Monday due to temperatures below -40°F.",
    "The Department of Motor Vehicles will be closed for system maintenance this Saturday, December 7.",
    "A boil water advisory has been issued for residents in the Eagle River area due to a water main break.",
    "The annual property tax payment deadline is December 31st. Payments can be made online or at City Hall.",
    "State parks will offer free admission on the first Saturday of every month through April.",
    "A wildfire is approaching the Palmer area. Residents in zones A and B are ordered to evacuate immediately."
]

# Generate responses using both prompt versions
print("Generating responses for evaluation dataset...")
responses_v1 = []
responses_v2 = []

for i, announcement in enumerate(announcements_dataset):
    print(f"  Processing announcement {i+1}/{len(announcements_dataset)}...", end="")
    r_v1 = generate_government_social_media_post(announcement, prompt_version="v1")
    r_v2 = generate_government_social_media_post(announcement, prompt_version="v2")
    responses_v1.append(r_v1)
    responses_v2.append(r_v2)
    print(" done")

print("\n✅ Response generation complete.")

# Preview the responses side by side
df_preview = pd.DataFrame({
    "Announcement": [a[:60] + "..." for a in announcements_dataset],
    "V1 Response": [r[:80] + "..." for r in responses_v1],
    "V2 Response": [r[:80] + "..." for r in responses_v2],
})
print("\n=== Response Preview ===")
print(df_preview.to_string(index=False))

### Step C2: Build Evaluation DataFrames

The Evaluation API requires a pandas DataFrame with specific column names.

In [ ]:
# -------------------------------------------------------
# Build Eval DataFrames for Prompt V1 and V2
# -------------------------------------------------------

# The full prompt (instruction + announcement) becomes the 'content' and 'instruction'
full_prompts_v1 = [SOCIAL_MEDIA_PROMPT_V1.format(announcement=a) for a in announcements_dataset]
full_prompts_v2 = [SOCIAL_MEDIA_PROMPT_V2.format(announcement=a) for a in announcements_dataset]

# The announcement alone serves as 'context' (grounding reference)
contexts = announcements_dataset

# Evaluation DataSet — Prompt V1
eval_dataset_v1 = pd.DataFrame({
    "content":    full_prompts_v1,
    "instruction": full_prompts_v1,
    "context":    contexts,
    "response":   responses_v1,    # Pre-computed responses (avoids re-running model in eval)
})

# Evaluation DataSet — Prompt V2
eval_dataset_v2 = pd.DataFrame({
    "content":    full_prompts_v2,
    "instruction": full_prompts_v2,
    "context":    contexts,
    "response":   responses_v2,
})

print(f"✅ Eval dataset V1: {len(eval_dataset_v1)} rows")
print(f"✅ Eval dataset V2: {len(eval_dataset_v2)} rows")
print("\nDataset V1 columns:", list(eval_dataset_v1.columns))
eval_dataset_v1.head(3)

### Step C3: Create EvalTask and Run Evaluation

In [ ]:
# -------------------------------------------------------
# Metrics to evaluate
# -------------------------------------------------------
EVAL_METRICS = ["fluency", "coherence", "fulfillment", "groundedness"]

# Unique experiment name (Vertex AI Experiments must be globally unique per project)
EXPERIMENT_NAME = "govt-social-media-post-eval"

# -------------------------------------------------------
# EvalTask for Prompt V1
# -------------------------------------------------------
eval_task_v1 = EvalTask(
    dataset=eval_dataset_v1,
    metrics=EVAL_METRICS,
    experiment=EXPERIMENT_NAME,
)

# -------------------------------------------------------
# EvalTask for Prompt V2
# -------------------------------------------------------
eval_task_v2 = EvalTask(
    dataset=eval_dataset_v2,
    metrics=EVAL_METRICS,
    experiment=EXPERIMENT_NAME,
)

print("✅ EvalTask objects created for V1 and V2.")

In [ ]:
# -------------------------------------------------------
# Run Evaluation — Prompt V1
# -------------------------------------------------------
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print("Running evaluation for Prompt V1 (basic prompt)...")
result_v1 = eval_task_v1.evaluate(
    model=model,
    experiment_run_name=f"social-media-v1-{run_ts}"
)
print("✅ V1 evaluation complete.")

In [ ]:
# -------------------------------------------------------
# Run Evaluation — Prompt V2
# -------------------------------------------------------
run_ts_v2 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print("Running evaluation for Prompt V2 (few-shot prompt)...")
result_v2 = eval_task_v2.evaluate(
    model=model,
    experiment_run_name=f"social-media-v2-{run_ts_v2}"
)
print("✅ V2 evaluation complete.")

### Step C4: Display and Compare Results

In [ ]:
def display_eval_report(label: str, result) -> None:
    """
    Pretty-prints evaluation summary metrics and the per-row metrics table.

    Args:
        label (str): A human-readable label for the evaluation run.
        result: The EvalResult object returned by EvalTask.evaluate().
    """
    print(f"\n{'='*60}")
    print(f"  EVALUATION REPORT: {label}")
    print(f"{'='*60}")
    print("\n📊 Summary Metrics:")
    print(result.summary_metrics.to_string())
    print("\n📋 Per-Row Metrics Table (first 5 rows):")
    print(result.metrics_table.head(5).to_string())
    print()


display_eval_report("Prompt V1 (Basic)", result_v1)
display_eval_report("Prompt V2 (Few-Shot Examples)", result_v2)

### Step C5: Side-by-Side Comparison

In [ ]:
# -------------------------------------------------------
# Head-to-Head Comparison: V1 vs V2
# -------------------------------------------------------

metrics_to_compare = ["fluency/mean", "coherence/mean", "fulfillment/mean", "groundedness/mean"]

comparison_data = {}
for metric in metrics_to_compare:
    v1_val = result_v1.summary_metrics.get(metric, "N/A")
    v2_val = result_v2.summary_metrics.get(metric, "N/A")
    comparison_data[metric] = {
        "Prompt V1 (Basic)": round(v1_val, 3) if isinstance(v1_val, float) else v1_val,
        "Prompt V2 (Few-Shot)": round(v2_val, 3) if isinstance(v2_val, float) else v2_val,
        "Winner": "V2" if (isinstance(v2_val, float) and isinstance(v1_val, float) and v2_val > v1_val)
                  else ("V1" if (isinstance(v1_val, float) and isinstance(v2_val, float) and v1_val > v2_val)
                        else "Tie")
    }

df_comparison = pd.DataFrame(comparison_data).T

print("\n" + "="*60)
print("  PROMPT V1 vs V2 — Head-to-Head Comparison")
print("="*60)
print(df_comparison.to_string())
print()

# Count wins
v2_wins = (df_comparison["Winner"] == "V2").sum()
v1_wins = (df_comparison["Winner"] == "V1").sum()

print(f"\n🏆 Overall: V1 wins {v1_wins} metric(s), V2 wins {v2_wins} metric(s)")
if v2_wins > v1_wins:
    print("✅ Recommendation: Use Prompt V2 (few-shot examples) for better quality social media posts.")
elif v1_wins > v2_wins:
    print("✅ Recommendation: Prompt V1 (basic) performs comparably — keep it simple.")
else:
    print("✅ Recommendation: Both prompts perform similarly. V2 provides more predictable output formatting.")

---
## Summary

### What Was Built

| Component | Description |
|---|---|
| `classify_citizen_question()` | Gemini-powered question classifier (4 categories) |
| `generate_government_social_media_post()` | Gemini-powered social media post generator (2 prompt versions) |
| `does_post_follow_rules()` | LLM-based rules checker for indeterminate outputs |
| `TestClassifyCitizenQuestion` | 9 pytest unit tests for classification (deterministic) |
| `TestGenerateGovernmentSocialMediaPost` | 5 pytest tests using LLM judge (indeterminate) |
| Google Evaluation API | Evaluated V1 vs V2 on fluency, coherence, fulfillment, groundedness |

### Key Takeaways

1. **Deterministic LLM functions** (classification, sentiment analysis) can be tested like any regular function — assert the exact output.
2. **Indeterminate LLM functions** (text generation) require an LLM-as-judge approach for unit testing.
3. **The Google Evaluation API** provides model-based quality metrics that go beyond pass/fail to measure real-world output quality.
4. **Few-shot prompting** (V2) typically improves quality metrics by demonstrating the expected output format to the model.